In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import tree
import graphviz
from sklearn import metrics

In [3]:
df=pd.read_excel('C:\\Users\\Surendra\\OneDrive\\Desktop\\train_s3TEQDk.xlsx')
df.set_index('ID',inplace=True)
df.head()
dfvalidation =pd.read_excel('C:\\Users\\Surendra\\OneDrive\\Desktop\\test_mSzZ8RL.xlsx')
dfvalidation .head()
dfvalidation .set_index('ID',inplace=True)
dfvalidation['Is_Lead'] = 1

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
ID,,,,,,,,,,
NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0
IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0
HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0
TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0


,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NaN,925537,No
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No
3,TT8RPZVC,Male,29,RG272,Other,X1,33,No,868070,No
4,SHQZEYTZ,Female,29,RG270,Other,X1,19,No,657087,No


In [4]:
##Merging the train and test dataset
dfmerged = pd.concat([df,dfvalidation])
dfmerged.shape

(351037, 10)

In [5]:
dfmerged.isna().sum()

Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         41847
Avg_Account_Balance        0
Is_Active                  0
Is_Lead                    0
dtype: int64

In [6]:
dfmerged['Credit_Product'].describe()

count     309190
unique         2
top           No
freq      205965
Name: Credit_Product, dtype: object

In [7]:
dfmerged.columns

Index(['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code', 'Vintage',
       'Credit_Product', 'Avg_Account_Balance', 'Is_Active', 'Is_Lead'],
      dtype='object')

In [8]:
## Handling missing values
dfmerged['Credit_Product']=dfmerged.Credit_Product.fillna(dfmerged['Credit_Product'].mode()[0])

In [9]:
dfmerged['Credit_Product'] = dfmerged['Credit_Product'].map({'Yes':1,'No':0})
dfmerged['Is_Active'] = dfmerged['Is_Active'].map({'Yes':1,'No':0})

In [10]:
##One hot encoding
dfmerged = pd.concat([dfmerged[['Age','Vintage','Credit_Product', 'Avg_Account_Balance', 'Is_Active', 'Is_Lead']],
               pd.get_dummies(dfmerged['Gender'],drop_first = True),
                      pd.get_dummies(dfmerged['Occupation'],drop_first = True),
                      pd.get_dummies(dfmerged['Channel_Code'],drop_first = True),
                      pd.get_dummies(dfmerged['Region_Code'],drop_first = True)],axis=1)

In [11]:
dfmerged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 351037 entries, NNVBBKZB to F2NOYPPZ
Data columns (total 47 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   Age                  351037 non-null  int64
 1   Vintage              351037 non-null  int64
 2   Credit_Product       351037 non-null  int64
 3   Avg_Account_Balance  351037 non-null  int64
 4   Is_Active            351037 non-null  int64
 5   Is_Lead              351037 non-null  int64
 6   Male                 351037 non-null  uint8
 7   Other                351037 non-null  uint8
 8   Salaried             351037 non-null  uint8
 9   Self_Employed        351037 non-null  uint8
 10  X2                   351037 non-null  uint8
 11  X3                   351037 non-null  uint8
 12  X4                   351037 non-null  uint8
 13  RG251                351037 non-null  uint8
 14  RG252                351037 non-null  uint8
 15  RG253                351037 non-null  uint8
 16

In [12]:
###Generating all possible pair of interactions between 2 pair of columns.
##Then removing those columns 

from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

Y = dfmerged.Is_Lead
X = dfmerged.drop(['Is_Lead'],1)
 


def add_interactions(df):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns)+['_'.join(x) for x in combos]
    
    #scaler = MinMaxScaler()
    #scaler.fit(df)
    #df = scaler.transform(df)
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indices = [i for i,x in enumerate(list((df==0).all())) if x]
    df= df.drop(df.columns[noint_indices], axis=1)
    
    return df
X = add_interactions(X)
X.shape

(351037, 514)

In [13]:
###Seperate out the training and testing dataset

X.index = Y.index
X['Is_Lead'] = Y

X_validation = X[245725:]
X = X[0:245725]

In [14]:
X_validation.shape

(105312, 515)

In [15]:
Y = X.Is_Lead
X = X.drop(['Is_Lead'],1)

In [16]:
from sklearn.model_selection import train_test_split
seed = 3
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [17]:
###Hypertune the model intensively with a 5 fold cross validation strategy.
##first grid search for max_depth,min_child_weight then fix those and search for rest in same fashion
### parameter scale_pos_weight is quite important in case of imbalanced dataset

from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn import preprocessing
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV


param_test2b = {
#    'nthread':[4], #when use hyperthread, xgboost may become slower
    'objective':['binary:logistic'],
    'learning_rate': [0.1], #so called `eta` value
    'max_depth':[5],
    'min_child_weight':[1],
    'subsample':[0.8],
    'colsample_bytree':[0.8],
    'n_estimators':[150],
    'scale_pos_weight':[3],
    'gamma':[0],
    'seed': [27]
}

xgb_model = XGBClassifier()

gsearch2b = GridSearchCV(xgb_model, param_test2b, n_jobs = 5, cv=5, 
                         scoring='roc_auc',verbose=2, refit=True, iid=False)

#gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=150, max_depth=5,
#                                                   min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
#                                                   objective= 'binary:logistic', nthread=4, scale_pos_weight=3,seed=27), 
#                         param_grid = param_test2b, scoring='f1',n_jobs=4,iid=False, cv=5)

gsearch2b.fit(X_train, y_train)
#print(gsearch2b.grid_scores_)
print("gsearch2b.best_params_",gsearch2b.best_params_)
print("gsearch2b.best_score_",gsearch2b.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 23.4min remaining: 35.1min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 23.6min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 23.6min finished
C:\Users\Surendra\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\Users\Surendra\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:25:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameters=None,
                                     verbosity=None),
             iid=False, n_jobs=5,
             param_grid={'colsample_by

gsearch2b.best_params_ {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 150, 'objective': 'binary:logistic', 'scale_pos_weight': 3, 'seed': 27, 'subsample': 0.8}
gsearch2b.best_score_ 0.7841568036511386


In [21]:
###Train the model with the best params
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, 
                      gamma=0.4,nthread=4, subsample=0.8, colsample_bytree=0.8,
                        objective= 'binary:logistic',scale_pos_weight=3,seed=29)
modelXg.fit(X_train, y_train)
y_xg = modelXg.predict(X_test)

[14:35:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=7, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=29, reg_alpha=0, reg_lambda=1, scale_pos_weight=3,
              seed=29, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [22]:
## Determine whether your model is overfitting or not , with the help of ROC.
print(confusion_matrix(y_test, y_xg))
predictions = [value for value in y_xg]
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
print("precision_score: %.2f%% on test dataset" % (precision * 100.0))
print("recall_score: %.2f%% on test dataset" % (recall * 100.0))
print("f1_score: %.2f%% on test dataset" % (f1 * 100.0))
print("roc_auc test set", roc_auc_score(y_test, modelXg.predict_proba(X_test)[:,1]))
print("roc_auc training set", roc_auc_score(y_train, modelXg.predict_proba(X_train)[:,1]))

[[39991 16199]
 [ 4846 12682]]
Accuracy_score: 71.45% on test dataset
precision_score: 43.91% on test dataset
recall_score: 72.35% on test dataset
f1_score: 54.65% on test dataset
roc_auc test set 0.7847093337513257
roc_auc training set 0.7951453941546422


In [23]:
##Once the model is ready, train the model on entire dataset.
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, 
                      gamma=0.4,nthread=4, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)

modelXg.fit(X, Y)

C:\Users\Surendra\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:41:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=7, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=29, reg_alpha=0, reg_lambda=1, scale_pos_weight=3,
              seed=29, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [24]:
#X = X_validation.drop(['Is_Lead'],1)
dfvalidation = X_validation.drop(['Is_Lead'],1)
y_valid = modelXg.predict(dfvalidation)
submission = dfvalidation.copy()
submission['Is_Lead'] = y_valid
submission['Is_Lead'].value_counts()

0    63945
1    41367
Name: Is_Lead, dtype: int64

In [25]:
submission = submission[['Is_Lead']]
submission.reset_index(inplace=True)
submission.to_csv("C:\\Users\\Surendra\\OneDrive\\Desktop\\\\solution\\solution_3.csv",index=False)

In [ ]:
pwd

In [27]:
predict_test = gsearch2b.predict(dfvalidation)

In [28]:
submission = dfvalidation.copy()
submission['Is_Lead'] = predict_test
submission['Is_Lead'].value_counts()
submission = submission[['Is_Lead']]
submission.reset_index(inplace=True)
submission.to_csv("C:\\Users\\Surendra\\OneDrive\\Desktop\\\\solution\\solution_4.csv",index=False)

0    64478
1    40834
Name: Is_Lead, dtype: int64

In [29]:
## predicted probability

predicted_prob = gsearch2b.predict_proba(dfvalidation)
np.round(predicted_prob,3)

array([[0.893, 0.107],
       [0.532, 0.468],
       [0.795, 0.205],
       ...,
       [0.416, 0.584],
       [0.24 , 0.76 ],
       [0.814, 0.186]], dtype=float32)

In [30]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(class_weight='balanced',random_state=23)


In [45]:
my_estimators = [10,100,1000]
my_max_depth = [5,10,15]
my_criterion = ['gini','entropy']
my_min_samples_leaf = [2,5,10,15]
my_min_samples_split = [2,5,10]
## Building the model

# Cross validation tells how well the model performs on a dataset using multiple samples of train data
grid_rf = GridSearchCV(estimator = forest,cv=4,scoring='roc_auc',
                   param_grid = dict(n_estimators=n_estimators,max_depth = max_depth, criterion=criterion,
                                    min_samples_leaf = min_samples_leaf,
                                    min_samples_split = min_samples_split))

In [46]:
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=4,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=23),
             param_grid={'criterion': ['entropy'], 'max_depth': [5],
                         'min_samples_leaf': [10], 'min_samples_split': [2],
                         'n_estimators': [100]},
             scoring='roc_auc')

In [47]:
predict_test = gsearch2b.predict(dfvalidation)

In [48]:
submission = dfvalidation.copy()
submission['Is_Lead'] = predict_test
submission['Is_Lead'].value_counts()
submission = submission[['Is_Lead']]
submission.reset_index(inplace=True)
submission.to_csv("C:\\Users\\Surendra\\OneDrive\\Desktop\\\\solution\\solution_5.csv",index=False)

0    64478
1    40834
Name: Is_Lead, dtype: int64